In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, chdir
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent.parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)

import numpy as np
from random import randint
from math import floor, ceil
import codecs

In [ ]:
from bulletin import root, default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils import utils, static
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import trim_overspace
from bulletin.utils.normalize import normalize_text, normalize_cpf#normalize_hash, normalize_labels, , date_hash, normalize_number

In [ ]:
from datetime import datetime, date, timedelta

exclusao_pathfile = join(root, 'database', 'casos_confirmados')

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

In [ ]:
from datetime import datetime

In [ ]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'].apply(normalize_text) + '/' + municipios['uf']

In [ ]:
### load
notifica = Notifica()
notifica.databases()
notifica.load('notifica',compress=False)

In [ ]:
# trabalhador_educacao = notifica.df.loc[(notifica.df['situacao_trabalho'].isin([4,5])) & (notifica.df['ibge_residencia'] == 410690) & (notifica.df['classificacao_final'] == 2)].copy()

In [ ]:
# trabalhador_educacao = trabalhador_educacao.loc[trabalhador_educacao['cod_cbo'].astype('str').str.startswith('23')][['id', 'data_diagnostico', 'cod_cbo', 'situacao_trabalho', 'classificacao_final', 'evolucao']]

In [ ]:
# trabalhador_educacao = trabalhador_educacao.merge(how = 'left', right = notifica.tables['cbo'].rename(columns={'codigo':'cod_cbo', 'id':'ident'}), on='cod_cbo')

In [ ]:
# trabalhador_educacao[['id', 'data_diagnostico', 'cod_cbo', 'valor', 'situacao_trabalho', 'classificacao_final', 'evolucao']]

In [ ]:
########### OUTROS TESTES

In [ ]:
cc = CasosConfirmados()
cc.load(f"cc_{ontem.strftime('%d_%m_%Y')}", compress=True)

In [ ]:
cc.df

In [ ]:
trab_1 = pd.read_excel(join(default_input, 'servidores_educacao_1.xlsx'), dtype={'CPF':str})

In [ ]:
trab_2 = pd.read_excel(join(default_input, 'servidores_educacao_2.xlsx'), dtype={'CPF':str})

In [ ]:
trabalhadores_educacao = pd.concat([trab_1, trab_2])
trabalhadores_educacao

In [ ]:
trabalhadores_educacao['CPF'] = trabalhadores_educacao['CPF'].apply(normalize_cpf)

In [ ]:
trabalhadores_educacao = trabalhadores_educacao.rename(columns={'CPF':'cpf'})
trabalhadores_educacao

In [ ]:
trabalhadores_educacao_infectados = pd.merge(cc.df[['id_notifica', 'ibge_residencia', 'ibge_unidade_notifica', 'paciente', 'cpf', 'sexo', 'idade', 'data_diagnostico', 'evolucao', 'data_cura_obito']], trabalhadores_educacao[['cpf', 'NOME']], on='cpf', how='inner')

In [ ]:
trabalhadores_educacao_infectados